In [14]:
import zipfile, requests
import torch

# Download HG38 data

In [13]:
!mkdir dataset
# !curl https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/GCF_000001405.40/download?include_annotation_type=GENOME_FASTA --output 'dataset/HG38.zip'

# datasets
hg38 = 'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/GCF_000001405.40/download'
t2t = 'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/GCF_009914755.1/download'

response = requests.get(hg38, params={'include_annotation_type': 'GENOME_FASTA'})
if response.status_code == 200:
    with open('dataset/HG38.zip', 'wb') as f:
        f.write(response.content)

zipfile.ZipFile('dataset/HG38.zip', 'r').extractall(path='dataset/')
print("dataset ready!")

gh38_fasta = 'dataset/ncbi_dataset/data/GCF_000001405.40/GCF_000001405.40_GRCh38.p14_genomic.fna'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  927M    0  927M    0     0  9712k      0 --:--:--  0:01:37 --:--:-- 17.4M55k-:-- 6738k
download complete!


# Model

In [3]:
print("CUDA is available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
gpu = 4
torch.cuda.set_device(gpu)
print("Current GPU:", torch.cuda.get_device_name())

CUDA is available: True
GPU count: 8
Current GPU: NVIDIA RTX 6000 Ada Generation
